<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-Web-scraping-for-CopernicusEMS-service" data-toc-modified-id="Python-Web-scraping-for-CopernicusEMS-service-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python Web scraping for CopernicusEMS service</a></span></li></ul></div>

# Python Web scraping for CopernicusEMS service

The goal of this document is to automatize the download of vector files provided from the CopernicusEMS services and their aggregation in a unique merged file.

This procedure has to be followed if you upload merged files that you already created from CopernicusEMS.

Import Beautiful Soup

In [1]:
from bs4 import BeautifulSoup as BS

In [2]:
import urllib
import urllib2

In [3]:
import requests

Set the base URL to the relevant CopernicusEMS component

In [4]:
baseurl = "http://emergency.copernicus.eu/mapping/list-of-components/EMSR332/"

Create a soup object with the baseurl

In [5]:
soup = BS(urllib2.urlopen(baseurl))

/usr/lib/python2.7/dist-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Extract all links (using the "< a >" tags where the attribute "href" is valid)

In [6]:
links = soup.findAll('a', href=True)

Preparation for the operation of merging.

In [7]:
import os
import zipfile
import shutil
import glob
import shapefile

###############################################################################

folder_input_zip = "./files/zipinput/"
folder_output_zip = "./files/zipoutput/"
array_id = ["_01","_02","_03","_04","_05","_06","_07","_08","_09"]

folder_input_shp = folder_output_zip
folder_output_shp = "./files/shapefiles/"
array_elements = ['observed_event']
folder_input_merge = folder_output_shp
folder_output_merge = "./files/merged/"

##############################################################################

Check and print which zipfiles are already stored in the "./files/zipinput/" directory

In [11]:
zipfiles = glob.glob("./files/zipinput/*.zip")
zipnames = []
for f in zipfiles:
    f = f.split('/')[-1]
    zipnames.append(f)
zipnames.sort()
zipnames
#fname = zipurltot.split('/')[-1]

['EMSR332_01VERONA_01DELINEATION_MAP_v1_vector.zip',
 'EMSR332_01VERONA_01DELINEATION_MONIT01_v1_vector.zip',
 'EMSR332_02PADOVA_01DELINEATION_MAP_v1_vector.zip',
 'EMSR332_02PADOVA_01DELINEATION_MONIT01_v1_vector.zip',
 'EMSR332_03ROVERETO_01DELINEATION_MAP_v1_vector.zip',
 'EMSR332_04BASSANODELGRAPPA_01DELINEATION_MAP_v1_vector.zip',
 'EMSR332_04BASSANODELGRAPPA_01DELINEATION_MONIT01_v1_vector.zip',
 'EMSR332_05TRENTO_01DELINEATION_MAP_v1_vector.zip',
 'EMSR332_06FELTRE_01DELINEATION_MAP_v1_vector.zip',
 'EMSR332_07CORTINEDAMPEZZO_01DELINEATION_MAP_v1_vector.zip',
 'EMSR332_08PORDENONE_01DELINEATION_MAP_v1_vector.zip',
 'EMSR332_08PORDENONE_01DELINEATION_MONIT01_v1_vector.zip',
 'EMSR332_09VENEZIA_01DELINEATION_MAP_v1_vector.zip',
 'EMSR332_09VENEZIA_01DELINEATION_MONIT01_v1_vector.zip']

Check if there are new files in the Copernicus activation and, if present, download them.

In [12]:
files_new = []
for link in links:
    zipurl = link['href']
    if( zipurl.endswith('.zip') ):
        zipurltot = 'http://emergency.copernicus.eu/' + zipurl
        fname = zipurltot.split('/')[-1]
        r = requests.get(zipurltot, stream=True)
        if fname not in zipnames:
            print fname
            files_new.append(fname)
            if( r.status_code == requests.codes.ok ):
                ! curl -s -c cookie $zipurltot
                ! curl -b -s cookie $zipurltot -H "Origin: http://emergency.copernicus.eu" -H "Accept-Encoding: gzip, deflate" -H "Accept-Language: it-IT,it;q=0.8,en-US;q=0.6,en;q=0.4" -H "Upgrade-Insecure-Requests: 1" -H "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36" -H "Content-Type: application/x-www-form-urlencoded" -H "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8" -H "Cache-Control: max-age=0" -H "Referer: zipurltot" -H "Connection: keep-alive" --data "confirmation=1^&op=+Download+file+^&form_build_id=form-wfWtSFuhPbanIpxEiVM8LPHnvLF5LEOuakUYLcXkCeI^&form_id=emsmapping_disclaimer_download_form" --compressed > ./files/zipinput/$fname

List newly downloaded files (if any)

In [13]:
files_new

[]

In [ ]:
def extract_zipped_files (file_name):
    #pass
    print file_name
    file_name_in=folder_input_zip+file_name
    with zipfile.ZipFile(file_name_in, "r") as z:
        z.extractall(folder_output_zip)

def clean_folder ():
    #pass
    folder_del = folder_output_zip + "*.*"
    files = sorted(glob.glob(folder_del))
    for f in files:
        os.remove(f)
    for element in array_elements:
        folder_del = folder_output_shp + element + "/*.*"
        files = sorted(glob.glob(folder_del))
        for f in files:
            os.remove(f)
    for element in array_elements:
        folder_del = folder_output_merge + element + "_merged.*"
        files = sorted(glob.glob(folder_del))
        for f in files:
            os.remove(f)


clean_folder()
#exit()

Extract files from zip and copy into a new folder.

In [ ]:
for file in sorted(os.listdir(folder_input_zip)):
    for id in array_id:
        if id in file:
            if file.endswith(".zip"):
                extract_zipped_files(file)

Sometimes there are zip files containing a folder named "VECTOR" so that shapefiles are one level below. The following code moves the files inside the VECTOR folder back to the "zipoutput" folder.

In [ ]:
for folder in os.walk('./files/zipoutput/').next()[1]:
    if folder == "VECTOR":
        print "VECTOR"
        vector = "./files/zipoutput/VECTOR/" + "*.*"
        back = "./files/zipoutput/"
        files = sorted(glob.glob(vector))
        for f in files:
            shutil.move(f,back)
        !rmdir "./files/zipoutput/VECTOR"

Move files in aggregated folders per type of shapefile.

In [ ]:
for file in sorted(os.listdir(folder_input_shp)):
    for element in array_elements:
        if element in file:
            file_move_name_in = folder_input_shp + file
            file_move_name_out = folder_output_shp + element + "/" + file
            shutil.move(file_move_name_in,file_move_name_out)

Merge

In [ ]:
for category in array_elements:
    folder_input_in = folder_input_merge + category + '/*.shp'
    folder_input_out = folder_output_merge + category + '_merged.shp'
    files = sorted(glob.glob(folder_input_in))
    w = shapefile.Writer()
    for f in files:
        r = shapefile.Reader(f)
        w._shapes.extend(r.shapes())
        w.records.extend(r.records())
        w.fields = list(r.fields)
    w.save(folder_input_out)
    folder_input_in = folder_input_merge + category + '/*.prj'
    folder_input_out = folder_output_merge + category + '_merged.prj'
    files = sorted(glob.glob(folder_input_in))
    for f in files:
        shutil.copy(f,folder_input_out)
        break

print "Finito !"